In [3]:
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")

import numpy as np
from visu import Visualizer
import imageio


# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches

from PIL import Image
import cv2
import torch
from torchvision import transforms as tf
from utils_asl import LabelLoaderAuto
import numpy as np
from pathlib import Path
import copy
import numpy as np
import pickle


lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")

def get_labels(p,sub=1,la="png"):
    p = [str(s) for s in Path(p).rglob(f'*.{la}') if str(s).find("_.png") == -1]
    
    p.sort(
        key=lambda x: int(x.split("/")[-1][:-4])
    )
    
    return p[::sub]

def get_labels_multiple( base_name, scenes_list,sub=1, test=False, train=False):
    labels = []
    for s in scenes_list:
        labels += get_labels(base_name.replace("xxxx",s) ,sub=sub)
    
    if test: 
        return labels[int( len(labels)*0.8):]
    
    if train:
        return labels[:int( len(labels)*0.8)]
    
    return labels



class AccMonitor():
    def __init__(self):
        self.metrics = {}
        self.metric_dict = {'correct': 0,
                            'correct_valid': 0,
                            'correct_valid_both': 0,
                            'total': 0, 
                            'total_valid':0,
                            'total_valid_both':0,
                            'images': 0}
    def register(self,name):
        self.metrics[name] = copy.deepcopy(self.metric_dict)
        
    def update(self,name, label, gt):
        if name not in self.metrics.keys():
            self.register(name)
        m1 = gt != -1
        self.metrics[name]["total_valid"] += (m1).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m1] == label[m1])
        
        m2 = (gt != -1 )*(label != -1)
        self.metrics[name]["total_valid_both"] += (m2).sum()
        self.metrics[name]["correct_valid_both"] += np.sum(gt[m2] == label[m2])
        
        self.metrics[name]["total"] += gt.size
        self.metrics[name]["correct"] += np.sum(gt == label)
        self.metrics[name]["images"] += 1
    
    def store_pickle(self, p):
        with open(p, 'wb') as handle:
            pickle.dump(self.metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def restore_pickle(self, p):
        with open(p, 'rb') as handle:
            self.metrics = pickle.load(handle)
            
    def __str__(self):
        s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s

In [2]:
import torch
import numpy as np
import os

from sklearn.metrics import confusion_matrix

def nanmean(data, **args):
    # This makes it ignore the first 'background' class
    return np.ma.masked_array(data, np.isnan(data)).mean(**args)
    # In np.ma.masked_array(data, np.isnan(data), elements of data == np.nan is invalid and will be ingorned during computation of np.mean()


class SemanticsMeter:
    def __init__(self, number_classes):
        self.conf_mat = None
        self.number_classes = number_classes

    def clear(self):
        self.conf_mat = None

    def prepare_inputs(self, *inputs):
        outputs = []
        for i, inp in enumerate(inputs):
            if torch.is_tensor(inp):
                inp = inp.detach().cpu().numpy()
            outputs.append(inp)

        return outputs

    def update(self, preds, truths):
        preds, truths = self.prepare_inputs(preds, truths) # [B, N, 3] or [B, H, W, 3], range[0, 1]
        preds = preds.flatten()
        truths = truths.flatten()
        valid_pix_ids = truths!=-1
        preds = preds[valid_pix_ids] 
        truths = truths[valid_pix_ids]
        conf_mat_current = confusion_matrix(truths, preds, labels=list(range(self.number_classes)))
        if self.conf_mat is None:
            self.conf_mat = conf_mat_current
        else:
            self.conf_mat += conf_mat_current

    def measure(self):
        conf_mat = self.conf_mat
        norm_conf_mat = np.transpose(
            np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))

        missing_class_mask = np.isnan(norm_conf_mat.sum(1)) # missing class will have NaN at corresponding class
        exsiting_class_mask = ~ missing_class_mask

        class_average_accuracy = nanmean(np.diagonal(norm_conf_mat))
        total_accuracy = (np.sum(np.diagonal(conf_mat)) / np.sum(conf_mat))
        ious = np.zeros(self.number_classes)
        for class_id in range(self.number_classes):
            ious[class_id] = (conf_mat[class_id, class_id] / (
                    np.sum(conf_mat[class_id, :]) + np.sum(conf_mat[:, class_id]) -
                    conf_mat[class_id, class_id]))
        miou_valid_class = np.mean(ious[exsiting_class_mask])
        return miou_valid_class, total_accuracy, class_average_accuracy





In [6]:
gt_p = "/home/jonfrey/Datasets/scannet/scans/xxxx/label-filt"

data = []
data.append( [ 1, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx", "iter1_pred"])
data.append( [ 1, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_4_scenes_map_1/scans/xxxx" ,"iter1_repro" ])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay/scans/xxxx", "iter2_0_pred"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay_reprojected/scans/xxxx", "iter2_0_repro"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02/scans/xxxx", "iter2_02_pred"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02_reprojected/scans/xxxx", "iter2_02_repro"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay05/scans/xxxx", "iter2_05_pred"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay09/scans/xxxx", "iter2_9_pred"])

data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iteration_3_individual_replay02_reprojected/scans/xxxx" , "iter3_02_repro"])
data.append( [ 3, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iteration_3_individual_replay02/scans/xxxx" , "iter3_02_pred"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter1_5cm_reprojected/scans/xxxx" , "iter1_repro_5cm"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_gt_reprojected/scans/xxxx" , "gt_repro_5cm"])
data.append( [ 1, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_gt_reprojected_3cm/scans/xxxx" , "gt_repro_3cm"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay00/scans/xxxx" , "iter2_00_5cm_pred"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02/scans/xxxx" , "iter2_02_5cm_pred"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_rep02_reprojected/scans/xxxx" , "iter2_02_5cm_repro"])

data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx" , "iter2_02_20_epochs_5cm_pred"])
scenes = [["scene0000_00",
        "scene0000_01",
        "scene0000_02"],
        ["scene0001_00",
        "scene0001_01"],
        ["scene0002_00",
        "scene0002_01"],
        ["scene0003_00",
        "scene0003_01",
       "scene0003_02"],
        ["scene0004_00"],
        ["scene0005_00",
        "scene0005_01"]]

data_arr = np.array( data )


In [9]:
 data_arr

array([['1', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx',
        'iter1_pred'],
       ['1', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_4_scenes_map_1/scans/xxxx',
        'iter1_repro'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay/scans/xxxx',
        'iter2_0_pred'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay_reprojected/scans/xxxx',
        'iter2_0_repro'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02/scans/xxxx',
        'iter2_02_pred'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02_reprojected/scans/xxxx',
        'iter2_02_repro'],
       

In [19]:
def array_to_acc( data_arr, gt_p, scenes , mode = "test", force=True, clean=False):
    NR = data_arr.shape[0]
    
    
    if clean: 
        for i in range(NR):
            path = data_arr[i,3]+"_"+ mode +"_res.pkl"
            try:
                os.remove(path)
            except:
                pass
    if mode == "test":
        test = True
    else:
        test = False
    
    if mode == "train":
        train = True
    else:
        train = False
    
    acc = np.zeros( (NR, len(scenes)) )
    miou = np.zeros( (NR, len(scenes)) )
    
    for i in range(NR):
        path = data_arr[i,3]+"_"+ mode +"_res.pkl"
        print(path)
        if os.path.isfile(path) and not force:
                with open(path, 'rb') as handle:
                    acc[i] = pickle.load(handle)
                
        else:
        
            for nr_s, s in enumerate( scenes):
                accm = AccMonitor() 
                sm = SemanticsMeter(number_classes=40)
                sm.clear()
                print(data_arr[i,2])
                print(s)
                gt_paths = get_labels_multiple( gt_p, s ,10, test, train)
                ppp = get_labels_multiple( data_arr[i,2] , s ,1, test, train)
                print( len( ppp ) )
                if len( ppp) != len(gt_paths):
                    print("ERROR break")
                    return False
                
                
                for j, gtp in enumerate( gt_paths):
                    gt = lla.get( gtp )[0]
                    tmp = lla.get( ppp[j] )[0]
                    accm.update("acc", tmp-1 ,gt-1)
                    sm.update(tmp-1, gt-1)
                    
                    if j % 10 == 0 :
                       print(j, "/", len(gt_paths))
                
                miou_valid_class, total_accuracy, class_average_accuracy = sm.measure()
                acc[i,nr_s] = accm.metrics["acc"]['correct_valid'] / accm.metrics["acc"]['total_valid']
                miou[i,nr_s] = miou_valid_class
                print( "new", miou_valid_class, total_accuracy, class_average_accuracy )
                print( "old", acc[i,nr_s])
                
            with open(path.replace("pkl", "_acc.pkl"), 'wb') as handle:
                pickle.dump(acc[i], handle, protocol=pickle.HIGHEST_PROTOCOL)
                print(acc)
                
            with open(path.replace("pkl", "_iou.pkl"), 'wb') as handle:
                pickle.dump(miou[i], handle, protocol=pickle.HIGHEST_PROTOCOL)
                print(acc)

    return acc, miou

results_acc, miou = array_to_acc( data_arr, gt_p, scenes , "test", True, False)

iter1_pred_test_res.pkl
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx
['scene0000_00', 'scene0000_01', 'scene0000_02']
354
0 / 354
10 / 354
20 / 354
30 / 354
40 / 354
50 / 354
60 / 354
70 / 354
80 / 354
90 / 354
100 / 354
110 / 354
120 / 354
130 / 354
140 / 354
150 / 354
160 / 354
170 / 354
180 / 354
190 / 354
200 / 354
210 / 354
220 / 354
230 / 354
240 / 354
250 / 354
260 / 354
270 / 354
280 / 354
290 / 354
300 / 354
310 / 354
320 / 354
330 / 354
340 / 354
350 / 354


<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.18104485253753722 0.6049862463304617 0.2555931686356105
old 0.6049862463304617
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx
['scene0001_00', 'scene0001_01']
57
0 / 57
10 / 57
20 / 57
30 / 57
40 / 57
50 / 57


<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.24342181844042046 0.48509447823992996 0.3513438116057734
old 0.48509447823992996
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx
['scene0002_00', 'scene0002_01']
250
0 / 250
10 / 250
20 / 250
30 / 250
40 / 250
50 / 250
60 / 250
70 / 250
80 / 250
90 / 250
100 / 250
110 / 250
120 / 250
130 / 250
140 / 250
150 / 250
160 / 250
170 / 250
180 / 250
190 / 250
200 / 250
210 / 250
220 / 250
230 / 250
240 / 250


...

<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.4481367400106764 0.7993583676285042 0.5043700213782235
old 0.7993583676285042
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0004_00']
19
0 / 19
10 / 19


<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.27953141324947095 0.42328039291432074 0.3609669586334858
old 0.42328039291432074
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0005_00', 'scene0005_01']
53
0 / 53
10 / 53
20 / 53
30 / 53
40 / 53
50 / 53
new 0.2687850647502194 0.5218981168206713 0.3670688544307172
old 0.5218981168206713
[[0.60498625 0.48509448 0.30540719 0.7463797  0.40322308 0.46614244]
 [0.7020355  0.53726839 0.37969778 0.81589819 0.43985224 0.52694764]
 [0.71716119 0.51586213 0.36670945 0.81303485 0.40976906 0.54252611]
 [0.74437654 0.47021905 0.38687946 0.81519364 0.42128564 0.58080423]
 [0.71635012 0.52928841 0.3675483  0.80578344 0.42671741 0.54784689]
 [0.73598977 0.49441966 0.39387526 0.80948828 0.42291923 0.58031192]
 [0.72056896 0.49821505 0.36718885 0.78917493 0.42189349 0.54847653]
 [0.68250248 0.41731615 0.34956933 0.79232646 0.40955072 0.5252033 ]
 [0.73236467 0.40698352 0.3791079  0.80236693 0.39109449 0.57641194]
 [0.72067376 0.4

<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


In [17]:
H,W = 100, 100

sm = SemanticsMeter(number_classes=40)
# miou_valid_class, total_accuracy, class_average_accuracy 

pred = torch.ones( (H,W), dtype=torch.float32)


label= torch.ones( (H,W), dtype=torch.float32)
label[0,:] = -1
label[1:,:] = 1
# We only have a single class. The prediction is the same as the label. mIoU = 1

sm.update(pred, label)
print(sm.measure())
sm.clear()


label= torch.ones( (H,W), dtype=torch.float32)
label[0,:] = -1
label[1:50,:] = 0
# We have two classes
# class 0:
# TP = 0, -> IoU = 0

# class 1:
# TP = 50 , TP = 50 , FP = 49, FN = 0 -> IoU 50/99

# mIoU = 0 / 2 + (50/99)/2 = 0.25252525252


sm.update(pred, label)
print(sm.measure())
sm.clear()

label= torch.ones( (H,W), dtype=torch.float32)
label[:50,:] = 0
label[50:,:] = 1

sm.update(pred, label)
print(sm.measure())
sm.clear()

# We have two classes
# class 0:
# TP = 0, -> IoU = 0

# class 1:
# TP = 50 , TP = 50 , FP = 50, FN = 0 -> IoU 50/100

# mIoU = 0 / 2 + (50/100)/2 = 0.25



(1.0, 1.0, 1.0)
(0.25252525252525254, 0.5050505050505051, 0.5)
(0.25, 0.5, 0.5)


<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (
<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (
<ipython-input-2-e4fbe2c04070>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-e4fbe2c04070>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


In [20]:
# FOR IOU 
import pandas as pd
names = data_arr[:,-1].tolist()
df = pd.DataFrame(data= miou[:,:-1].T, columns=names)
df.mean()

d2 = np.zeros( (miou[:,:-1].shape[0],miou[:,:-1].shape[1]+1 ) )
d2[:,:-1] = miou[:,:-1]
d2[:,-1] = df.mean()

df2 = pd.DataFrame(data= d2.T, columns=names )

print("iter_1 == pretrained network output")
print("iter_1_reprojected == pseudo label reprojected")
print("cl -> network output;  XX percentage of replayed samples \n \n")

print( df.mean() )
#print( df.std() )

df2


iter_1 == pretrained network output
iter_1_reprojected == pseudo label reprojected
cl -> network output;  XX percentage of replayed samples 
 

iter1_pred                     0.238741
iter1_repro                    0.273283
iter2_0_pred                   0.263290
iter2_0_repro                  0.268135
iter2_02_pred                  0.266217
iter2_02_repro                 0.269805
iter2_05_pred                  0.260395
iter2_9_pred                   0.227039
iter3_02_repro                 0.244905
iter3_02_pred                  0.246282
iter1_repro_5cm                0.278680
gt_repro_5cm                   0.791360
gt_repro_3cm                   0.829283
iter2_00_5cm_pred              0.256987
iter2_02_5cm_pred              0.257557
iter2_02_5cm_repro             0.268252
iter2_02_20_epochs_5cm_pred    0.247014
dtype: float64


,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,0.181045,0.230824,0.224374,0.259439,0.212926,0.242349,0.227404,0.187809,0.234289,0.210605,0.251009,0.748023,0.788409,0.225012,0.215341,0.249495,0.212417
1,0.243422,0.249988,0.227853,0.199010,0.234610,0.213234,0.222634,0.182120,0.153146,0.189760,0.243984,0.882641,0.906009,0.182011,0.194683,0.200149,0.173434
2,0.114164,0.142540,0.137818,0.144707,0.136701,0.148912,0.136742,0.117963,0.130014,0.127091,0.133944,0.765467,0.838723,0.132113,0.125403,0.148381,0.121553
3,0.430283,0.469777,0.455685,0.451357,0.452691,0.449245,0.433805,0.426073,0.432119,0.432718,0.470531,0.852419,0.904884,0.454443,0.452702,0.442758,0.448137
4,0.224789,0.273287,0.270720,0.286162,0.294156,0.295284,0.281388,0.221228,0.274956,0.271235,0.293932,0.708250,0.708388,0.291357,0.299655,0.300475,0.279531
5,0.238741,0.273283,0.263290,0.268135,0.266217,0.269805,0.260395,0.227039,0.244905,0.246282,0.278680,0.791360,0.829283,0.256987,0.257557,0.268252,0.247014


In [21]:
# FOR IOU
(df2*100).round(1)



,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,18.1,23.1,22.4,25.9,21.3,24.2,22.7,18.8,23.4,21.1,25.1,74.8,78.8,22.5,21.5,24.9,21.2
1,24.3,25.0,22.8,19.9,23.5,21.3,22.3,18.2,15.3,19.0,24.4,88.3,90.6,18.2,19.5,20.0,17.3
2,11.4,14.3,13.8,14.5,13.7,14.9,13.7,11.8,13.0,12.7,13.4,76.5,83.9,13.2,12.5,14.8,12.2
3,43.0,47.0,45.6,45.1,45.3,44.9,43.4,42.6,43.2,43.3,47.1,85.2,90.5,45.4,45.3,44.3,44.8
4,22.5,27.3,27.1,28.6,29.4,29.5,28.1,22.1,27.5,27.1,29.4,70.8,70.8,29.1,30.0,30.0,28.0
5,23.9,27.3,26.3,26.8,26.6,27.0,26.0,22.7,24.5,24.6,27.9,79.1,82.9,25.7,25.8,26.8,24.7


In [8]:
# FOR ACC
import pandas as pd
names = data_arr[:,-1].tolist()
df = pd.DataFrame(data= results_acc[:,:-1].T, columns=names)
df.mean()

d2 = np.zeros( (results_acc[:,:-1].shape[0],results_acc[:,:-1].shape[1]+1 ) )
d2[:,:-1] = results_acc[:,:-1]
d2[:,-1] = df.mean()

df2 = pd.DataFrame(data= d2.T, columns=names )

print("iter_1 == pretrained network output")
print("iter_1_reprojected == pseudo label reprojected")
print("cl -> network output;  XX percentage of replayed samples \n \n")

print( df.mean() )
#print( df.std() )

df2

iter_1 == pretrained network output
iter_1_reprojected == pseudo label reprojected
cl -> network output;  XX percentage of replayed samples 
 

iter1_pred                     0.509018
iter1_repro                    0.574950
iter2_0_pred                   0.564507
iter2_0_repro                  0.567591
iter2_02_pred                  0.569138
iter2_02_repro                 0.571338
iter2_05_pred                  0.559408
iter2_9_pred                   0.530253
iter3_02_repro                 0.542384
iter3_02_pred                  0.552132
iter1_repro_5cm                0.577131
gt_repro_5cm                   0.925080
gt_repro_3cm                   0.934849
iter2_00_5cm_pred              0.548650
iter2_02_5cm_pred              0.553335
iter2_02_5cm_repro             0.563247
iter2_02_20_epochs_5cm_pred    0.541114
dtype: float64


,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,0.604986,0.702035,0.717161,0.744377,0.716350,0.735990,0.720569,0.682502,0.732365,0.720674,0.715534,0.942653,0.952927,0.725416,0.720906,0.736860,0.717174
1,0.485094,0.537268,0.515862,0.470219,0.529288,0.494420,0.498215,0.417316,0.406984,0.474861,0.534082,0.948616,0.950438,0.428223,0.455272,0.470114,0.407660
2,0.305407,0.379698,0.366709,0.386879,0.367548,0.393875,0.367189,0.349569,0.379108,0.367435,0.374544,0.887161,0.926547,0.363498,0.362313,0.381418,0.358097
3,0.746380,0.815898,0.813035,0.815194,0.805783,0.809488,0.789175,0.792326,0.802367,0.796995,0.813634,0.959801,0.976596,0.807545,0.801219,0.804275,0.799358
4,0.403223,0.439852,0.409769,0.421286,0.426717,0.422919,0.421893,0.409551,0.391094,0.400696,0.447860,0.887168,0.867738,0.418569,0.426968,0.423566,0.423280
5,0.509018,0.574950,0.564507,0.567591,0.569138,0.571338,0.559408,0.530253,0.542384,0.552132,0.577131,0.925080,0.934849,0.548650,0.553335,0.563247,0.541114


In [6]:
(df2*100).round(1)

,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro
0,60.5,70.2,71.7,74.4,71.6,73.6,72.1,68.3,73.2,72.1,71.6,94.3,95.3,72.5,72.1,73.7
1,48.5,53.7,51.6,47.0,52.9,49.4,49.8,41.7,40.7,47.5,53.4,94.9,95.0,42.8,45.5,47.0
2,30.5,38.0,36.7,38.7,36.8,39.4,36.7,35.0,37.9,36.7,37.5,88.7,92.7,36.3,36.2,38.1
3,74.6,81.6,81.3,81.5,80.6,80.9,78.9,79.2,80.2,79.7,81.4,96.0,97.7,80.8,80.1,80.4
4,40.3,44.0,41.0,42.1,42.7,42.3,42.2,41.0,39.1,40.1,44.8,88.7,86.8,41.9,42.7,42.4
5,50.9,57.5,56.5,56.8,56.9,57.1,55.9,53.0,54.2,55.2,57.7,92.5,93.5,54.9,55.3,56.3


In [ ]:
visu = Visualizer( p_visu= "/home/jonfrey/Pictures/test")    
    
gt_paths = get_labels_multiple( gt_p, ["scene0000_00"] ,10, False,False)
ppp = get_labels_multiple( data_arr[i,2] , ["scene0000_00"] ,1, False,False)

In [ ]:
target = "labels_iter2_5cm_replay02"
base = "/home/jonfrey/Datasets/labels_generated/" + target + "/scans"

for s in [x for x in os.listdir(base)]:
    print(s)
    b =  os.path.join(base,s)
    bb = os.listdir( b)[0]
    src = os.path.join(b,bb)
    dst = os.path.join(b,target)
    print( src, dst)
    os.rename(src, dst)

In [ ]:
import shutil 
target = "iteration_2_individual_replay09"
base = "/home/jonfrey/Datasets/labels_generated/" + target
for s in [x[0] for x in os.walk(base) if x[0].split("/")[-1][:7] ==  "scene00"] :
    dst = os.path.join(base, "scans")
    
    try:
        pass
        #shutil.move(s, dst)
    except:
        pass